In [2]:
import uproot
import pandas as pd
import numpy as np

In [3]:
tfile = uproot.open('electron_features.root')
mc = pd.DataFrame(
      tfile['features/tree'].arrays(
         tfile['features/tree'].keys()
         )
      )

In [4]:
mc[mc.is_e][['gen_pt','trk_pt','gsf_pt','ele_pt','preid_trk_ecal_Deta']].head()

,gen_pt,trk_pt,gsf_pt,ele_pt,preid_trk_ecal_Deta
0,3.608217,3.363398,3.456851,3.416700,0.057574
1,3.100642,3.081321,3.155636,3.641827,0.003723
2,3.924682,-1.000000,-1.000000,-1.000000,-1.000000
3,12.313822,12.179570,11.894906,14.609233,-0.000939
4,1.070635,1.024662,1.094570,-1.000000,0.006878


In [5]:
missing = mc.is_e & (mc.gsf_pt > 0) & (mc.ele_pt <= 0)
miss = mc[missing]
trk_dr = np.sqrt((miss.gen_eta - miss.trk_eta)**2 + (miss.gen_phi- miss.trk_phi)**2)
gsf_dr = np.sqrt((miss.gen_eta - miss.gsf_eta)**2 + (miss.gen_phi- miss.gsf_phi)**2)

In [6]:
miss[miss.gen_pt > 4][['lumi','evt','gen_pt','trk_pt','gsf_pt','ele_pt','preid_trk_ecal_Deta']].head()

,lumi,evt,gen_pt,trk_pt,gsf_pt,ele_pt,preid_trk_ecal_Deta
13,106572,31971353,5.426744,5.531412,1.736093,-1.0,-0.003527
83,197294,59188092,5.626984,3.422086,3.397850,-1.0,0.011344
99,198241,59472208,4.315388,4.142424,4.623238,-1.0,-0.011715
163,201513,60453732,4.425927,4.392927,5.440060,-1.0,1000.000000
222,203639,61091548,17.000290,17.094954,17.432041,-1.0,-0.001104


In [8]:
[i for i in miss.columns if 'has' in i]

['has_ele_core',
 'has_pfBlock',
 'has_pfBlock_with_SC',
 'has_pfEgamma',
 'has_pfGSF_trk']

In [21]:
mask = (miss.preid_trk_ecal_Deta < 999) & np.invert(miss.has_ele_core)
#has_pfGSF_trk has_pfEgamma
mask1 = (mask & miss.has_pfGSF_trk &np.invert(miss.has_pfEgamma))
to_check1 = miss[mask1][['run','lumi','evt','gsf_pt','gsf_eta','gsf_phi']]

print ','.join([':'.join([str(j) for j in i]) for i in to_check1[['run','lumi','evt']].as_matrix()])
print '\n\n'
vv = {}
for i in range(to_check1.shape[0]):
    entry = to_check1.iloc[i]
    vv[int(entry.evt)] = (entry.gsf_pt, entry.gsf_eta, entry.gsf_phi)
print vv
to_check1.head()
print to_check1.shape[0]

1:188451:56535285,1:197582:59274487,1:198749:59624562,1:201111:60333114,1:202082:60624397,1:203454:61035956,1:204665:61399352,1:205841:61752053



{60624397: (3.2524049282073975, 0.37861114740371704, -2.5944802761077881), 59624562: (1.3961746692657471, -1.7653611898422241, 0.35166314244270325), 61035956: (1.3295170068740845, -0.74432915449142456, -0.89448297023773193), 56535285: (2.057009220123291, -0.78594261407852173, -2.8598525524139404), 59274487: (1.2078419923782349, -0.71046125888824463, 1.0331467390060425), 61399352: (1.9714157581329346, -1.4073941707611084, 1.7577246427536011), 60333114: (1.0912861824035645, -0.87859433889389038, -2.2439939975738525), 61752053: (1.9641132354736328, -1.4127223491668701, 2.0960731506347656)}
8


In [22]:
#has_pfGSF_trk has_pfEgamma
mask2 = (mask & np.invert(miss.has_pfGSF_trk) & (miss.gsf_pt > 0))
to_check2 = miss[mask2][['run','lumi','evt','gsf_pt','gsf_eta','gsf_phi']]

print ','.join([':'.join([str(j) for j in i]) for i in to_check2[['run','lumi','evt']].as_matrix()])
print '\n\n'
vv = {}
for i in range(to_check2.shape[0]):
    entry = to_check2.iloc[i]
    vv[int(entry.evt)] = (entry.gsf_pt, entry.gsf_eta, entry.gsf_phi)
print vv
print to_check2.shape[0]
to_check2.head()

1:106498:31949309,1:196220:58865712,1:198241:59472208,1:201071:60321251,1:203639:61091548,1:204605:61381254,1:205812:61743594,1:205952:61785330,1:190534:57160055,1:192724:57817001,1:194675:58402401



{59472208: (4.6232380867004395, -0.65895694494247437, -0.25869864225387573), 60321251: (1.1779265403747559, 0.82428324222564697, 1.5455923080444336), 61381254: (4.7505941390991211, 0.25948464870452881, -2.0914435386657715), 58402401: (1.7245339155197144, 0.44584566354751587, -1.5300849676132202), 57817001: (9.2789793014526367, 2.2519600391387939, 0.67897021770477295), 61743594: (9.8303279876708984, 0.39349210262298584, 2.9180049896240234), 58865712: (2.4383711814880371, -1.5381476879119873, 3.0827889442443848), 61785330: (1.6721429824829102, -0.74857711791992188, -1.1605315208435059), 57160055: (2.4560666084289551, 0.96155643463134766, -2.9944767951965332), 61091548: (17.432041168212891, -0.23632617294788361, 2.0985651016235352), 31949309: (3.1733708381652832, 1.6469196081161499, -1.10840

,run,lumi,evt,gsf_pt,gsf_eta,gsf_phi
9,1,106498,31949309,3.173371,1.646920,-1.108405
43,1,196220,58865712,2.438371,-1.538148,3.082789
99,1,198241,59472208,4.623238,-0.658957,-0.258699
151,1,201071,60321251,1.177927,0.824283,1.545592
222,1,203639,61091548,17.432041,-0.236326,2.098565


In [17]:
mask = (miss.gen_pt > 4)
to_check = miss[mask]
print ','.join([':'.join([str(j) for j in i]) for i in to_check[['run','lumi','evt']].as_matrix()])


1:106572:31971353,1:197294:59188092,1:198241:59472208,1:201513:60453732,1:203639:61091548,1:204605:61381254,1:205249:61574621,1:205812:61743594,1:189477:56843034,1:189942:56982449,1:192009:57602444,1:192724:57817001,1:195261:58578240



{60624397.0: (3.2524049282073975, 0.37861114740371704, -2.5944802761077881), 59624562.0: (1.3961746692657471, -1.7653611898422241, 0.35166314244270325), 61035956.0: (1.3295170068740845, -0.74432915449142456, -0.89448297023773193), 56535285.0: (2.057009220123291, -0.78594261407852173, -2.8598525524139404), 59274487.0: (1.2078419923782349, -0.71046125888824463, 1.0331467390060425), 61399352.0: (1.9714157581329346, -1.4073941707611084, 1.7577246427536011), 60333114.0: (1.0912861824035645, -0.87859433889389038, -2.2439939975738525), 61752053.0: (1.9641132354736328, -1.4127223491668701, 2.0960731506347656)}


In [25]:
oth_tf = uproot.open('PFTest.root')
oth = pd.DataFrame(
      oth_tf['features/tree'].arrays(
         oth_tf['features/tree'].keys()
         )
      )

In [26]:
oth[oth.gen_pt > 4][['lumi','evt','gen_pt','trk_pt','gsf_pt','ele_pt','preid_trk_ecal_Deta']].head()

,lumi,evt,gen_pt,trk_pt,gsf_pt,ele_pt,preid_trk_ecal_Deta
1,106572,31971353,5.426744,5.531412,1.736093,-1.0,-0.003527
475,189477,56843034,5.807426,5.726750,6.248170,-1.0,-0.004182
696,189942,56982449,5.732317,5.682927,5.785018,-1.0,-0.007494


In [28]:
set(oth.evt)

{31971353, 56843034, 56982449}

In [34]:
oth[oth.gen_pt > 4][['gsf_pt','gsf_eta','gsf_phi','ele_pt','has_ele_core']]

,gsf_pt,gsf_eta,gsf_phi,ele_pt,has_ele_core
1,1.736093,1.285409,-0.986466,-1.0,True
475,6.248170,1.087226,0.974058,-1.0,True
696,5.785018,-0.464799,0.657538,-1.0,True
